In [4]:
import re
import os
import time
import json
import PyPDF2
import pypandoc
import pandas as pd
from tqdm import tqdm
import lxml.etree as ET


from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

## Utility functions

In [5]:
isLinux = False
default_linux_path = os.getcwd() + "/work/"
default_windows_path = os.getcwd().replace("\\Data", "\\Documents\\Downloaded\\")
default_path = default_linux_path if isLinux else default_windows_path

DEFAULT_SAVE_DIR = default_path.replace("\\Downloaded", "\\Generated")
CHROME_DRIVERS_WINDOWS_PATH = "C:\\Users\\giaco\\Downloads\\chromedriver-win64\\chromedriver.exe"

# Utility functions and constants
def write_to_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read()

# Different kind of text extraction from each type of file
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text    

def extract_text_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    return ET.tostring(root, encoding='unicode', method='text')

def extract_text_from_rtf(rtf_path):
    return pypandoc.convert_file(rtf_path, 'plain', format='rtf')


def split_text(text, pattern):
    parts = re.split(pattern, text, flags=re.MULTILINE)
    
    parts = [part for part in parts if part]
    
    if not re.match(pattern, parts[0]):
        parts = parts[1:]
    
    return parts

def split_text(text, max_chunk_size=7000, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(separators=[
        "\n\n",
        "\n",
        ".",
    ],
    chunk_size=max_chunk_size,
    chunk_overlap=chunk_overlap)
    
    return text_splitter.split_text(text)

## Extraction of Codice Penale (from website of Procura Generale Trento)

In [3]:
CODICE_PENALE_DIR = DEFAULT_SAVE_DIR + "Codice penale well formatted edited.pdf"

def find_articles(text):
    # Remove all sections starting with "LIBRO"
    text = re.sub(r'LIBRO.*?(?=Articolo n\.|$)', '', text, flags=re.DOTALL)

    pattern = r'(Articolo n\..*?)(\n.*?)(?=\nArticolo n\.|$)'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

text = extract_text_from_pdf(CODICE_PENALE_DIR)

matches = find_articles(text)

data = []
for article in matches:
    law_number = article[0].split("Articolo n.")[1] # Get everything after "Articolo n."
    law_text = article[1].strip()
    if '.' in law_text:
        law_title, law_text = map(str.strip, law_text.split('.', 1))
    else:
        law_title = ''
    data.append({'Law number': law_number, 'Law title': law_title, 'Law text': law_text})

df = pd.DataFrame(data)
df.to_csv(os.getcwd() + "/work/documents/Codice_Penale.csv", index=False)

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\giaco\\source\\repos\\Thesis\\Documents\\Generated\\Codice penale well formatted edited.pdf'

## Extraction of C.P.P. from Normattiva ()

## Extraction of Dlgs from Normattiva

In [17]:
class NormattivaScraper:
    def __init__(self, driver_path, headless=True):
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        self.service = Service(driver_path)
        self.driver = webdriver.Chrome(service=self.service, options=chrome_options)
    
    # Get the originario version of the law
    def fill_field(self, field_id, value):
        input_field = self.driver.find_element(By.ID, field_id)
        input_field.clear()
        input_field.send_keys(value)
    
    # Get the text of a specific article
    def get_article_text(self, numeroProvvedimento, anno, article_num=[]):
        articles_list = []
            
        self.fill_field("numeroProvvedimento", numeroProvvedimento)
        self.fill_field("annoProvvedimento", anno)

        self.driver.find_element(By.CSS_SELECTOR, "[type*='submit']").click()
        self.driver.find_elements(By.CSS_SELECTOR, "[title*='Dettaglio atto']")[0].click()
        
        time.sleep(2)
        # Ensure is multivigente version
        multivigente_button = self.driver.find_element(By.XPATH, '//a[contains(@href, "multivigenza")]')
        multivigente_button.click()
        
        # Get articles
        albero = self.driver.find_element(By.ID, "albero")
        articles = albero.find_elements(By.CLASS_NAME, "numero_articolo")
        print("Article len: ", len(articles))
                
        for article in articles:            
            if article.text.strip() != "" and article.text[0].isdigit() and "orig" not in article.text and "Allegato" not in article.text and "agg" not in article.text:
                print("In ", article.text)
                try:
                    article.click()
                except:
                    continue
                
                time.sleep(2)
                
                article_title = self.driver.find_element(By.CLASS_NAME, "article-num-akn").text.strip()
                commas = self.driver.find_elements(By.CLASS_NAME, "art-comma-div-akn")
                print("Comma len: ", len(commas))
                if len(commas) == 0:
                    continue
                
                firstTime = True                
                for i, comma in enumerate(commas):
                    if firstTime:
                        time.sleep(1)
                        firstTime = False
                    print(numeroProvvedimento, anno, article_title, i)
                    
                    # Check if the content contains any multivigenza change
                    comma_content = comma.get_attribute('outerHTML')
                    #print(comma_content)
                    
                    # Skip if the content contains "<span class="art_text_in_comma">((</span>" or "<span class="art_text_in_comma">))</span>"
                    if "<span class=\"art_text_in_comma\">((</span>" in comma_content or "<span class=\"art_text_in_comma\">))</span>" in comma_content:
                        continue
                    
                    try:
                        comma_number = comma.find_element(By.CLASS_NAME, "comma-num-akn").text.strip()
                    except:
                        continue
                    comma_content_element = comma.text#find_element(By.CLASS_NAME, "art_text_in_comma")

                    #print(comma_number, comma_content)
                    
                    # Clear the output
                    match =  re.search(r'<span class="art_text_in_comma">(.*?)</span>', comma_content, re.DOTALL)
                    comma_content = match.group(1) if match else comma_content
                    comma_content = re.sub(r"<div class=\"ins-akn\" eid=\"ins_\d+\">\(\(", "", comma_content, flags=re.DOTALL)
                    comma_content = re.sub(r"\)\)</div>", "", comma_content, flags=re.DOTALL)
                    comma_content = re.sub(r"\n", "", comma_content, flags=re.DOTALL)
                    comma_content = re.sub(r"<br>", "", comma_content, flags=re.DOTALL)
                    
                    articles_list.append({ "Dlgs":f"{numeroProvvedimento}/{anno}".strip(),
                                            "Article": article_title,
                                            "Comma number": comma_number,
                                            "Comma content": comma_content.strip()}) # Numeration not working in case of -bis... extract
            else:
                print("Out ", article.text)

        return articles_list

    # Navigate to a specific page
    def navigate_to_page(self, url):
        self.driver.get(url)
        
    # Close the driver
    def close(self):
        self.driver.quit()

def save_articles(articles, filename):
    df = pd.DataFrame(articles)
    df.to_csv(filename, index=False)

scraper = NormattivaScraper(CHROME_DRIVERS_WINDOWS_PATH, headless=True)

# Check if the laws have already been scraped
scraped_laws_set = set()
if os.path.exists(DEFAULT_SAVE_DIR + '\\laws.csv'):
    df = pd.read_csv(DEFAULT_SAVE_DIR + '\\laws.csv')
    first_column = df.iloc[:, 0]
    scraped_laws = df.to_dict('records')
    
    # put the already scraped laws in a set to not scrape them again
    scraped_laws_set = set(first_column)
else:
    scraped_laws = []

invalid_laws_set = set()
if os.path.exists(DEFAULT_SAVE_DIR + '\\invalid_laws.json'):
    invalid_laws = json.loads(DEFAULT_SAVE_DIR + '\\invalid_laws.json')
    invalid_laws_set = set(invalid_laws)
else:
    invalid_laws = []

scraped_laws_set.update(invalid_laws_set)

df = pd.read_csv(DEFAULT_SAVE_DIR + '\\references_merged.csv')
articles = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]): # 445
    # Check if it's a D. lgs.
    if '/' not in row['Source'] or row['Source'].strip() in scraped_laws_set:
        continue
    scraped_laws_set.add(row['Source'])
    print(f"Scraping |{row['Source']}|")
    num, year = row['Source'].split("/")
            
    scraper.navigate_to_page("https://www.normattiva.it/ricerca/avanzata")
    res = scraper.get_article_text(num, year)
    if res == []:
        invalid_laws.append(f"{num}/{year}")
    else:
        articles.append(res)
    
    if articles and len(articles) % 3 == 0:
        save_articles([item for sublist in articles for item in sublist] + scraped_laws, DEFAULT_SAVE_DIR + '\\laws.csv')
        json.dumps(scraped_laws)

save_articles([item for sublist in articles for item in sublist] + scraped_laws, DEFAULT_SAVE_DIR + '\\laws.csv')
with open(DEFAULT_SAVE_DIR + '\\invalid_laws.json', 'w') as f:
    json.dumps(scraped_laws)

  0%|          | 0/972 [00:00<?, ?it/s]

Scraping | 50/2016|
Article len:  841
In  1
Comma len:  0
Out  
Out  
Out  
In  2
Comma len:  0
Out  
Out  
In  3
Comma len:  0
Out  
Out  
Out  
In  4
Comma len:  0
Out  
Out  
In  5
Comma len:  0
Out  
Out  
In  6
Comma len:  0
Out  
Out  
In  7
Comma len:  0
Out  
In  8
Comma len:  0
Out  
In  9
Comma len:  0
Out  
In  10
Comma len:  0
Out  
In  11
Comma len:  0
Out  
In  12
Comma len:  0
Out  
In  13
Comma len:  0
Out  
In  14
Comma len:  0
Out  
Out  
In  15
Comma len:  0
Out  
In  16
Comma len:  0
Out  
Out  
In  17
Comma len:  0
Out  
Out  
In  17 bis
Comma len:  0
Out  
In  18
Comma len:  0
Out  
Out  
In  19
Comma len:  0
Out  
In  20
Comma len:  0
Out  
In  21
Comma len:  0
Out  
Out  
In  22
Comma len:  0
Out  
Out  
Out  
In  23
Comma len:  0
Out  
Out  
Out  
Out  
Out  
In  24
Comma len:  0
Out  
Out  
Out  
In  25
Comma len:  0
Out  
Out  
Out  
Out  
In  26
Comma len:  0
Out  
Out  
Out  
Out  
In  27
Comma len:  0
Out  
Out  
Out  
In  28
Comma len:  0
Out  
Out  
Out 

 46%|████▌     | 448/972 [07:59<09:20,  1.07s/it]

Out  Allegato XXIV
Out  
Out  Allegato XXV
Out  
Scraping | 50/2016|
Article len:  841
In  1
Comma len:  0
Out  
Out  
Out  
In  2
Comma len:  0
Out  
Out  
In  3
Comma len:  0
Out  
Out  
Out  
In  4
Comma len:  0
Out  
Out  
In  5
Comma len:  0
Out  
Out  
In  6
Comma len:  0
Out  
Out  
In  7
Comma len:  0
Out  
In  8
Comma len:  0
Out  
In  9
Comma len:  0
Out  
In  10
Comma len:  0
Out  
In  11
Comma len:  0
Out  
In  12
Comma len:  0
Out  
In  13
Comma len:  0
Out  
In  14
Comma len:  0
Out  
Out  
In  15
Comma len:  0
Out  
In  16
Comma len:  0
Out  
Out  
In  17
Comma len:  0
Out  
Out  
In  17 bis
Comma len:  0
Out  
In  18
Comma len:  0
Out  
Out  
In  19
Comma len:  0
Out  
In  20
Comma len:  0
Out  
In  21
Comma len:  0
Out  
Out  
In  22
Comma len:  0
Out  
Out  
Out  
In  23
Comma len:  0
Out  
Out  
Out  
Out  
Out  
In  24
Comma len:  0
Out  
Out  
Out  
In  25
Comma len:  0
Out  
Out  
Out  
Out  
In  26
Comma len:  0
Out  
Out  
Out  
Out  
In  27
Comma len:  0
Out  


 47%|████▋     | 458/972 [15:54<21:27,  2.51s/it]

...

Out  Allegato XXIV
Out  
Out  Allegato XXV
Out  
Scraping | 50/2016|
Article len:  841
In  1
Comma len:  0
Out  
Out  
Out  
In  2
Comma len:  0
Out  
Out  
In  3
Comma len:  0
Out  
Out  
Out  
In  4
Comma len:  0
Out  
Out  
In  5
Comma len:  0
Out  
Out  
In  6
Comma len:  0
Out  
Out  
In  7
Comma len:  0
Out  
In  8
Comma len:  0
Out  
In  9
Comma len:  0
Out  
In  10
Comma len:  0
Out  
In  11
Comma len:  0
Out  
In  12
Comma len:  0
Out  
In  13
Comma len:  0
Out  
In  14
Comma len:  0
Out  
Out  
In  15
Comma len:  0
Out  
In  16
Comma len:  0
Out  
Out  
In  17
Comma len:  0
Out  
Out  
In  17 bis
Comma len:  0
Out  
In  18
Comma len:  0
Out  
Out  
In  19
Comma len:  0
Out  
In  20
Comma len:  0
Out  
In  21
Comma len:  0
Out  
Out  
In  22
Comma len:  0
Out  
Out  
Out  
In  23
Comma len:  0
Out  
Out  
Out  
Out  
Out  
In  24
Comma len:  0
Out  
Out  
Out  
In  25
Comma len:  0
Out  
Out  
Out  
Out  
In  26
Comma len:  0
Out  
Out  
Out  
Out  
In  27
Comma len:  0
Out  


 50%|████▉     | 484/972 [55:36<3:05:11, 22.77s/it]

Out  Allegato XVIII
Out  
Out  Allegato XIX
Out  
Out  Allegato XX
Out  
Out  Allegato XXI
Out  
Out  Allegato XXII
Out  
Out  Allegato XXIII
Out  
Out  Allegato XXIV
Out  
Out  Allegato XXV
Out  
Scraping | 50/2016|
Article len:  841
In  1
Comma len:  0
Out  
Out  
Out  
In  2
Comma len:  0
Out  
Out  
In  3
Comma len:  0
Out  
Out  
Out  
In  4
Comma len:  0
Out  
Out  
In  5
Comma len:  0
Out  
Out  
In  6
Comma len:  0
Out  
Out  
In  7
Comma len:  0
Out  
In  8
Comma len:  0
Out  
In  9
Comma len:  0
Out  
In  10
Comma len:  0
Out  
In  11
Comma len:  0
Out  
In  12
Comma len:  0
Out  
In  13
Comma len:  0
Out  
In  14
Comma len:  0
Out  
Out  
In  15
Comma len:  0
Out  
In  16
Comma len:  0
Out  
Out  
In  17
Comma len:  0
Out  
Out  
In  17 bis
Comma len:  0
Out  
In  18
Comma len:  0
Out  
Out  
In  19
Comma len:  0
Out  
In  20
Comma len:  0
Out  
In  21
Comma len:  0
Out  
Out  
In  22
Comma len:  0
Out  
Out  
Out  
In  23
Comma len:  0
Out  
Out  
Out  
Out  
Out  
In  24
C

 50%|████▉     | 485/972 [1:01:18<1:01:33,  7.58s/it]


ElementClickInterceptedException: Message: element click intercepted: Element <a href="" onclick="return showArticle('/atto/caricaArticolo?art.versione=5&amp;art.idGruppo=20&amp;art.flagTipoArticolo=0&amp;art.codiceRedazionale=16G00062&amp;art.idArticolo=...&amp;art.idSottoArticolo=1&amp;art.idSottoArticolo1=10&amp;art.dataPubblicazioneGazzetta=2016-04-19&amp;art.progressivo=0&amp;', this);" class="numero_articolo">157</a> is not clickable at point (212, 14). Other element would receive the click: <a href="javascript:" onclick="showArticle('/atto/caricaArticolo?art.versione=2&amp;art.idGruppo=20&amp;art.flagTipoArticolo=0&amp;art.codiceRedazionale=16G00062&amp;art.idArticolo=155&amp;art.idSottoArticolo=1&amp;art.idSottoArticolo1=10&amp;art.dataPubblicazioneGazzetta=2016-04-19 00:00:00.0&amp;art.progressivo=0&amp;riferimenti=false&amp;qId=&amp;art.imUpdate=false')" class="btn">...</a>
  (Session info: chrome=127.0.6533.89)
Stacktrace:
	GetHandleVerifier [0x00007FF617F5EEB2+31554]
	(No s